<a href="https://colab.research.google.com/github/soumik12345/mirnetv2/blob/main/notebooks/mirnetv2_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qq --upgrade wandb
!pip install -qq transformers tensorflow_addons

In [2]:
import os
import random
import numpy as np
from glob import glob
from functools import partial
from PIL import Image, ImageOps

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

import tensorflow as tf
import tensorflow_addons as tfa
from transformers.tf_utils import shape_list

import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint


AUTOTUNE = tf.data.AUTOTUNE

In [3]:
wandb.init(project="mirnet-v2", entity="ml-colabs", job_type="train")
config = wandb.config

config.seed = 42
random.seed(config.seed)
tf.random.set_seed(config.seed)

config.artifact_address = 'ml-colabs/mirnet-v2/new_dataset:v0'
config.image_size = 128
config.max_train_images = 400
config.batch_size = 4
config.initial_learning_rate = 2e-4
config.minimum_learning_rate = 1e-6
config.epochs = 300

artifact = wandb.use_artifact(config.artifact_address, type='dataset')
dataset_dir = artifact.download()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 19soumik-rakshit96 (ml-colabs). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact new_dataset:v0, 331.95MB. 1003 files... 
wandb:   1003 of 1003 files downloaded.  
Done. 0:0:0.2


In [4]:
TRAIN_LOW_LIGHT_IMAGES = sorted(
    glob(os.path.join(dataset_dir, "our485/low/*"))
)[:config.max_train_images]
TRAIN_GROUND_TRUTH_IMAGES = sorted(
    glob(os.path.join(dataset_dir, "our485/high/*"))
)[:config.max_train_images]

VAL_LOW_LIGHT_IMAGES = sorted(
    glob(os.path.join(dataset_dir, "our485/low/*"))
)[config.max_train_images:]
VAL_GROUND_TRUTH_IMAGES = sorted(
    glob(os.path.join(dataset_dir, "our485/high/*"))
)[config.max_train_images:]

TEST_LOW_LIGHT_IMAGES = sorted(
    glob(os.path.join(dataset_dir, "eval15/low/*"))
)
TEST_GROUND_TRUTH_IMAGES = sorted(
    glob(os.path.join(dataset_dir, "eval15/high/*"))
)

print(f"Train Dataset: ({len(TRAIN_LOW_LIGHT_IMAGES)}, {len(TRAIN_GROUND_TRUTH_IMAGES)})")
print(f"Validation Dataset: ({len(VAL_LOW_LIGHT_IMAGES)}, {len(VAL_GROUND_TRUTH_IMAGES)})")
print(f"Test Dataset: ({len(TEST_LOW_LIGHT_IMAGES)}, {len(TEST_GROUND_TRUTH_IMAGES)})")

Train Dataset: (400, 400)
Validation Dataset: (85, 85)
Test Dataset: (15, 15)


In [5]:
def read_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.cast(image, dtype=tf.float32) / 255.0
    return image


def random_crop(low_image, gt_image):
    low_image_shape = tf.shape(low_image)[:2]
    crop_width = tf.random.uniform(
        shape=(), maxval=low_image_shape[1] - config.image_size + 1, dtype=tf.int32
    )
    crop_height = tf.random.uniform(
        shape=(), maxval=low_image_shape[0] - config.image_size + 1, dtype=tf.int32
    )
    low_image_cropped = low_image[
        crop_height : crop_height + config.image_size,
        crop_width : crop_width + config.image_size
    ]
    gt_image_cropped = gt_image[
        crop_height : crop_height + config.image_size,
        crop_width : crop_width + config.image_size
    ]
    low_image_cropped.set_shape([config.image_size, config.image_size, 3])
    gt_image_cropped.set_shape([config.image_size, config.image_size, 3])
    return low_image_cropped, gt_image_cropped


def resize_images(low_image, gt_image):
    low_image = tf.image.resize(low_image, size=[config.image_size, config.image_size])
    gt_image = tf.image.resize(gt_image, size=[config.image_size, config.image_size])
    low_image.set_shape([config.image_size, config.image_size, 3])
    gt_image.set_shape([config.image_size, config.image_size, 3])
    return low_image, gt_image


def load_data(low_light_image_path, enhanced_image_path, apply_resize):
    low_light_image = read_image(low_light_image_path)
    enhanced_image = read_image(enhanced_image_path)
    low_light_image, enhanced_image = (
        resize_images(low_light_image, enhanced_image) if apply_resize
        else random_crop(low_light_image, enhanced_image)
    )
    return low_light_image, enhanced_image


def get_dataset(low_light_images, enhanced_images, apply_resize):
    dataset = tf.data.Dataset.from_tensor_slices((low_light_images, enhanced_images))
    dataset = dataset.map(
        partial(load_data, apply_resize=apply_resize),
        num_parallel_calls=AUTOTUNE
    )
    dataset = dataset.batch(config.batch_size, drop_remainder=True)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [6]:
train_dataset = get_dataset(TRAIN_LOW_LIGHT_IMAGES, TRAIN_GROUND_TRUTH_IMAGES, apply_resize=False)
val_dataset = get_dataset(VAL_LOW_LIGHT_IMAGES, VAL_GROUND_TRUTH_IMAGES, apply_resize=True)

print("Train Dataset:", train_dataset.element_spec)
print("Validation Dataset:", val_dataset.element_spec)

Train Dataset: (TensorSpec(shape=(4, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4, 128, 128, 3), dtype=tf.float32, name=None))
Validation Dataset: (TensorSpec(shape=(4, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4, 128, 128, 3), dtype=tf.float32, name=None))


## Model

### SKFF

In [7]:
class SelectiveKernelFeatureFusion(tf.keras.layers.Layer):
    def __init__(self, channels: int, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.hidden_channels = max(int(channels / 8), 4)

        self.average_pooling = tfa.layers.AdaptiveAveragePooling2D(output_size=1)

        self.conv_channel_downscale = tf.keras.layers.Conv2D(
            self.hidden_channels, kernel_size=1, padding="same"
        )
        self.conv_attention_1 = tf.keras.layers.Conv2D(
            channels, kernel_size=1, strides=1, padding="same"
        )
        self.conv_attention_2 = tf.keras.layers.Conv2D(
            channels, kernel_size=1, strides=1, padding="same"
        )

        self.leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)
        self.sorftmax = tf.keras.layers.Softmax(axis=-1)

    def call(self, inputs, *args, **kwargs):
        combined_input_features = inputs[0] + inputs[1]
        channel_wise_statistics = self.average_pooling(combined_input_features)
        downscaled_channel_wise_statistics = self.conv_channel_downscale(
            channel_wise_statistics
        )
        attention_vector_1 = self.sorftmax(
            self.conv_attention_1(downscaled_channel_wise_statistics)
        )
        attention_vector_2 = self.sorftmax(
            self.conv_attention_2(downscaled_channel_wise_statistics)
        )
        selected_features = (
            inputs[0] * attention_vector_1 + inputs[1] * attention_vector_2
        )
        return selected_features

### RCB

In [8]:
class ContextBlock(tf.keras.layers.Layer):
    def __init__(self, channels: int, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.mask_conv = tf.keras.layers.Conv2D(
            1, kernel_size=1, padding="same"
        )

        self.channel_add_conv_1 = tf.keras.layers.Conv2D(
            channels, kernel_size=1, padding="same"
        )
        self.channel_add_conv_2 = tf.keras.layers.Conv2D(
            channels, kernel_size=1, padding="same"
        )

        self.softmax = tf.keras.layers.Softmax(axis=1)
        self.leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)

    def modeling(self, inputs):
        _, height, width, channels = shape_list(inputs)
        reshaped_inputs = tf.expand_dims(
            tf.reshape(inputs, (-1, channels, height * width)), axis=1
        )

        context_mask = self.mask_conv(inputs)
        context_mask = tf.reshape(context_mask, (-1, height * width, 1))
        context_mask = self.softmax(context_mask)
        context_mask = tf.expand_dims(context_mask, axis=1)

        context = tf.reshape(
            tf.matmul(reshaped_inputs, context_mask), (-1, 1, 1, channels)
        )
        return context

    def call(self, inputs, *args, **kwargs):
        context = self.modeling(inputs)
        channel_add_term = self.channel_add_conv_1(context)
        channel_add_term = self.leaky_relu(channel_add_term)
        channel_add_term = self.channel_add_conv_2(channel_add_term)
        return inputs + channel_add_term


class ResidualContextBlock(tf.keras.layers.Layer):
    def __init__(self, channels: int, groups: int, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.conv_1 = tf.keras.layers.Conv2D(
            channels, kernel_size=3, padding="same", groups=groups
        )
        self.conv_2 = tf.keras.layers.Conv2D(
            channels, kernel_size=3, padding="same", groups=groups
        )
        self.leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)

        self.context_block = ContextBlock(channels=channels)

    def call(self, inputs):
        x = self.conv_1(inputs)
        x = self.leaky_relu(x)
        x = self.conv_2(x)
        x = self.context_block(x)
        x = self.leaky_relu(x)
        x = x + inputs
        return x

### Downsample

In [9]:
class DownBlock(tf.keras.layers.Layer):
    def __init__(
        self, channels: int, channel_factor: float, *args, **kwargs
    ):
        super(DownBlock, self).__init__(*args, **kwargs)
        self.average_pool = tf.keras.layers.AveragePooling2D(
            pool_size=2, strides=2
        )
        self.conv = tf.keras.layers.Conv2D(
            int(channels * channel_factor),
            kernel_size=1,
            strides=1,
            padding="same"
        )

    def call(self, inputs, *args, **kwargs):
        return self.conv(self.average_pool(inputs))


class DownSampleBlock(tf.keras.layers.Layer):
    def __init__(
        self,
        channels: int,
        scale_factor: int,
        channel_factor: float,
        *args,
        **kwargs
    ):
        super(DownSampleBlock, self).__init__(*args, **kwargs)
        self.layers = []
        for _ in range(int(np.log2(scale_factor))):
            self.layers.append(DownBlock(channels, channel_factor))
            channels = int(channels * channel_factor)

    def call(self, x, *args, **kwargs):
        for layer in self.layers:
            x = layer(x)
        return x

### Upsample

In [10]:
class UpBlock(tf.keras.layers.Layer):
    def __init__(self, channels: int, channel_factor: float, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.conv = tf.keras.layers.Conv2D(
            int(channels // channel_factor), kernel_size=1, strides=1, padding="same"
        )
        self.upsample = tf.keras.layers.UpSampling2D(size=2, interpolation="bilinear")

    def call(self, inputs, *args, **kwargs):
        return self.upsample(self.conv(inputs))


class UpSampleBlock(tf.keras.layers.Layer):
    def __init__(
        self, channels: int, scale_factor: int, channel_factor: float, *args, **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.layers = []
        for _ in range(int(np.log2(scale_factor))):
            self.layers.append(UpBlock(channels, channel_factor))
            channels = int(channels // channel_factor)

    def call(self, x, *args, **kwargs):
        for layer in self.layers:
            x = layer(x)
        return x

### MRB

In [11]:
class MultiScaleResidualBlock(tf.keras.layers.Layer):
    def __init__(
        self,
        channels: int,
        channel_factor: float,
        groups: int,
        *args,
        **kwargs
    ):
        super().__init__(*args, **kwargs)

        # Residual Context Blocks
        self.rcb_top = ResidualContextBlock(
            int(channels * channel_factor**0), groups=groups
        )
        self.rcb_middle = ResidualContextBlock(
            int(channels * channel_factor**1), groups=groups
        )
        self.rcb_bottom = ResidualContextBlock(
            int(channels * channel_factor**2), groups=groups
        )

        # Downsample Blocks
        self.down_2 = DownSampleBlock(
            channels=int((channel_factor**0) * channels),
            scale_factor=2,
            channel_factor=channel_factor,
        )
        self.down_4_1 = DownSampleBlock(
            channels=int((channel_factor ** 0) * channels),
            scale_factor=2,
            channel_factor=channel_factor,
        )
        self.down_4_2 = DownSampleBlock(
            channels=int((channel_factor ** 1) * channels),
            scale_factor=2,
            channel_factor=channel_factor,
        )

        # UpSample Blocks
        self.up21_1 = UpSampleBlock(
            channels=int((channel_factor ** 1) * channels),
            scale_factor=2,
            channel_factor=channel_factor,
        )
        self.up21_2 = UpSampleBlock(
            channels=int((channel_factor ** 1) * channels),
            scale_factor=2,
            channel_factor=channel_factor,
        )
        self.up32_1 = UpSampleBlock(
            channels=int((channel_factor ** 2) * channels),
            scale_factor=2,
            channel_factor=channel_factor,
        )
        self.up32_2 = UpSampleBlock(
            channels=int((channel_factor ** 2) * channels),
            scale_factor=2,
            channel_factor=channel_factor,
        )

        # SKFF Blocks
        self.skff_top = SelectiveKernelFeatureFusion(
            channels=int(channels * channel_factor**0)
        )
        self.skff_middle = SelectiveKernelFeatureFusion(
            channels=int(channels * channel_factor**1)
        )

        # Convolution
        self.conv_out = tf.keras.layers.Conv2D(
            channels, kernel_size=1, padding="same"
        )

    def call(self, inputs, *args, **kwargs):
        x_top = inputs
        x_middle = self.down_2(x_top)
        x_bottom = self.down_4_2(self.down_4_1(x_top))

        x_top = self.rcb_top(x_top)
        x_middle = self.rcb_middle(x_middle)
        x_bottom = self.rcb_bottom(x_bottom)

        x_middle = self.skff_middle([x_middle, self.up32_1(x_bottom)])
        x_top = self.skff_top([x_top, self.up21_1(x_middle)])

        x_top = self.rcb_top(x_top)
        x_middle = self.rcb_middle(x_middle)
        x_bottom = self.rcb_bottom(x_bottom)

        x_middle = self.skff_middle([x_middle, self.up32_2(x_bottom)])
        x_top = self.skff_top([x_top, self.up21_2(x_middle)])

        output = self.conv_out(x_top)
        output = output + inputs

        return output

### RRG

In [12]:
class RecursiveResidualGroup(tf.keras.layers.Layer):
    def __init__(
        self,
        channels: int,
        num_mrb_blocks: int,
        channel_factor: float,
        groups: int,
        *args,
        **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.layers = [
            MultiScaleResidualBlock(channels, channel_factor, groups)
            for _ in range(num_mrb_blocks)
        ]
        self.layers.append(
            tf.keras.layers.Conv2D(
                channels, kernel_size=3, strides=1, padding="same"
            )
        )

    def call(self, inputs, *args, **kwargs):
        residual = inputs
        for layer in self.layers:
            residual = layer(residual)
        residual = residual + inputs
        return residual

### MirNetV2

In [13]:
class MirNetv2(tf.keras.Model):
    def __init__(
        self,
        channels: int,
        channel_factor: float,
        num_mrb_blocks: int,
        add_residual_connection: bool,
        *args,
        **kwargs
    ):
        super().__init__(*args, **kwargs)

        self.add_residual_connection = add_residual_connection

        self.conv_in = tf.keras.layers.Conv2D(
            channels, kernel_size=3, padding="same"
        )

        self.rrg_block_1 = RecursiveResidualGroup(
            channels, num_mrb_blocks, channel_factor, groups=1
        )
        self.rrg_block_2 = RecursiveResidualGroup(
            channels, num_mrb_blocks, channel_factor, groups=2
        )
        self.rrg_block_3 = RecursiveResidualGroup(
            channels, num_mrb_blocks, channel_factor, groups=4
        )
        self.rrg_block_4 = RecursiveResidualGroup(
            channels, num_mrb_blocks, channel_factor, groups=4
        )

        self.conv_out = tf.keras.layers.Conv2D(
            3, kernel_size=3, padding="same"
        )

    def call(self, inputs, training=None, mask=None):
        shallow_features = self.conv_in(inputs)
        deep_features = self.rrg_block_1(shallow_features)
        deep_features = self.rrg_block_2(deep_features)
        deep_features = self.rrg_block_3(deep_features)
        deep_features = self.rrg_block_4(deep_features)
        output = self.conv_out(deep_features)
        output = output + inputs if self.add_residual_connection else output
        return output

In [14]:
model = MirNetv2(
    channels=80,
    channel_factor=1.5,
    num_mrb_blocks=2,
    add_residual_connection=True
)

dummy_inputs = tf.ones((1, config.image_size, config.image_size, 3))
dummy_outputs = model(dummy_inputs)
model.summary(expand_nested=True)
print("\nInput Shape:", dummy_inputs.shape)
print("Output Shape:", dummy_outputs.shape)

Model: "mir_netv2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  2240      
                                                                 
 recursive_residual_group (R  multiple                 2426936   
 ecursiveResidualGroup)                                          
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| multi_scale_residual_block   multiple               1184628   |
| (MultiScaleResidualBlock)                                     |
|                                                               |
| multi_scale_residual_block_  multiple               1184628   |
| 1 (MultiScaleResidualBlock)                                   |
|                                                               |
| conv2d_59 (Conv2D)        multiple                  57680     |
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯

## Losses and Metrics

In [15]:
class CharbonnierLoss(tf.keras.losses.Loss):
    def __init__(self, epsilon: float, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.epsilon = tf.convert_to_tensor(epsilon)

    def call(self, y_true, y_pred):
        squared_difference = tf.square(y_true - y_pred)
        return tf.reduce_mean(
            tf.sqrt(squared_difference + tf.square(self.epsilon))
        )


class PSNRMetric(tf.keras.metrics.Metric):
    def __init__(self, max_val: float, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.max_val = max_val
        self.psnr = tf.keras.metrics.Mean(name="psnr")

    def update_state(self, y_true, y_pred, *args, **kwargs):
        psnr = tf.image.psnr(y_true, y_pred, max_val=self.max_val)
        self.psnr.update_state(psnr, *args, **kwargs)

    def result(self):
        return self.psnr.result()

    def reset_state(self):
        self.psnr.reset_state()


class SSIMMetric(tf.keras.metrics.Metric):
    def __init__(self, max_val: float, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.max_val = max_val
        self.ssim = tf.keras.metrics.Mean(name="ssim")

    def update_state(self, y_true, y_pred, *args, **kwargs):
        ssim = tf.image.ssim(y_true, y_pred, max_val=self.max_val)
        self.ssim.update_state(ssim, *args, **kwargs)

    def result(self):
        return self.ssim.result()

    def reset_state(self):
        self.ssim.reset_state()

In [16]:
loss = CharbonnierLoss(epsilon=1e-3)

psnr_metric = PSNRMetric(max_val=1.0)
ssim_metric = SSIMMetric(max_val=1.0)

decay_steps = (config.max_train_images // config.batch_size) * config.epochs
lr_schedule_fn = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=config.initial_learning_rate,
    decay_steps=decay_steps,
    alpha=config.minimum_learning_rate,
)
optimizer = tf.keras.optimizers.Adam(
    learning_rate=lr_schedule_fn, beta_1=0.9, beta_2=0.999,
)

model.compile(
    optimizer=optimizer, loss=loss, metrics=[psnr_metric, ssim_metric]
)

In [ ]:
callbacks = [
    WandbMetricsLogger(),
    WandbModelCheckpoint(filepath="model", save_best_only=True)
]

model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=config.epochs,
    callbacks=callbacks
)

wandb: WARNING When using `save_best_only`, ensure that the `filepath` argument contains formatting placeholders like `{epoch:02d}` or `{batch:02d}`. This ensures correct interpretation of the logged artifacts.


Epoch 1/300
100/100 [==============================] - ETA: 0s - loss: 0.1717 - psnr_metric: 15.0998 - ssim_metric: 0.6540

wandb: Adding directory to artifact (./model)... Done. 0.6s


100/100 [==============================] - 170s 1s/step - loss: 0.1717 - psnr_metric: 15.0998 - ssim_metric: 0.6540 - val_loss: 0.1208 - val_psnr_metric: 17.9959 - val_ssim_metric: 0.7309
Epoch 2/300
100/100 [==============================] - ETA: 0s - loss: 0.1668 - psnr_metric: 15.6472 - ssim_metric: 0.6784

wandb: Adding directory to artifact (./model)... Done. 0.2s


100/100 [==============================] - 71s 710ms/step - loss: 0.1668 - psnr_metric: 15.6472 - ssim_metric: 0.6784 - val_loss: 0.1334 - val_psnr_metric: 16.4894 - val_ssim_metric: 0.7476
Epoch 3/300
100/100 [==============================] - ETA: 0s - loss: 0.1632 - psnr_metric: 15.6207 - ssim_metric: 0.6833

wandb: Adding directory to artifact (./model)... Done. 0.2s


100/100 [==============================] - 70s 700ms/step - loss: 0.1632 - psnr_metric: 15.6207 - ssim_metric: 0.6833 - val_loss: 0.1391 - val_psnr_metric: 16.6426 - val_ssim_metric: 0.7486
Epoch 4/300
 56/100 [===============>..............] - ETA: 28s - loss: 0.1730 - psnr_metric: 15.1493 - ssim_metric: 0.6616

In [ ]:
wandb.finish()